In [ ]:
import os
import json
import glob
import wandb
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader

# ==========================================
# 1. Setup & Auth
# ==========================================
WANDB_API_KEY = "api key"
PROJECT_NAME = "projectname"
ENTITY = "ENTITY"
wandb.login(key=WANDB_API_KEY)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amirbnsl (esi-sba-dz) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [12]:
# ==========================================
# 2. Helpers (No Download)
# ==========================================
class Cifar10DataManager:
    def __init__(self, data_dir="./data"):
        self.data_dir = data_dir
        self.mean = (0.4914, 0.4822, 0.4465)
        self.std = (0.2023, 0.1994, 0.2010)

    def get_loaders(self, batch_size, architecture_option='standard'):
        transform_list = [
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ]
        # Only test transform needed for eval
        test_transform = transforms.Compose(transform_list)
        if architecture_option == 'upsample':
            test_transform = transforms.Compose([transforms.Resize(224)] + transform_list)

        # STRICT: download=False
        test_set = torchvision.datasets.CIFAR10(root=self.data_dir, train=False, download=False, transform=test_transform)

        indices_path = os.path.join(self.data_dir, "processed", "test_indices.npy")
        real_test_set = Subset(test_set, np.load(indices_path))
        return DataLoader(real_test_set, batch_size=batch_size, shuffle=False)

def build_model(architecture_option='standard'):
    model = torchvision.models.resnet18(pretrained=True)
    if architecture_option == 'modified':
        model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        model.maxpool = nn.Identity()
    elif architecture_option == 'upsample':
        pass
    model.fc = nn.Linear(model.fc.in_features, 10)
    return model

In [4]:
# ==========================================
# 3. Fetch Artifacts
# ==========================================
run = wandb.init(project=PROJECT_NAME, entity=ENTITY, job_type="evaluation")


In [18]:

print("Downloading Data Artifact...")
run.use_artifact(f'{ENTITY}/{PROJECT_NAME}/cifar10_dataset:v0', type='dataset').download(root="./data")
artifact_dir = artifact.download()


wandb: Downloading large artifact 'cifar10_dataset:v0', 340.26MB. 11 files...
wandb:   11 of 11 files downloaded.  
Done. 00:00:03.9 (86.8MB/s)


In [13]:

# Resolve Model
api = wandb.Api()
sweeps = api.project(PROJECT_NAME, entity=ENTITY).sweeps()
sweep_id = sweeps[0].id if sweeps else os.getenv("SWEEP_ID")
best_run = api.sweep(f"{ENTITY}/{PROJECT_NAME}/{sweep_id}").best_run()
config = best_run.config

print(f"Downloading Model from: {best_run.name}")
model_dir = best_run.logged_artifacts()[0].download(root="./models")
model_path = glob.glob(os.path.join(model_dir, "*.pth"))[0]

wandb: Sorting runs by -summary_metrics.val_acc


wandb:   1 of 1 files downloaded.  


In [22]:
!ls ./artifacts/cifar10_dataset\:v0

cifar-10-batches-py  cifar-10-python.tar.gz  processed


In [24]:
# ==========================================
# 4. Evaluation
# ==========================================
dm = Cifar10DataManager(data_dir="./data")
test_loader = dm.get_loaders(100, config['architecture_option'])

model = build_model(config['architecture_option']).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        all_preds.extend(torch.max(outputs, 1)[1].cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

wandb.log({"confusion_matrix": wandb.plot.confusion_matrix(
    y_true=all_labels, preds=all_preds,
    class_names=['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
)})
run.finish()
print("Evaluation Complete.")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 120MB/s]


Evaluation Complete.
